In [1]:
import os
import time
import datetime
import numpy as np
import warnings
import random
from PIL import Image
from tqdm import tqdm

from labels import labels

In [2]:
# 데이터 경로
#  os.getcwd() :'/home/ag/Capstone2/DeepLab_V3_ver1'
origin_data_path = os.getcwd() + '/data/Original_data/'
labeled_data_path = os.getcwd() + '/data/Labeled_data/'

origin_data_list = os.listdir(origin_data_path) # x
labeled_data_list = os.listdir(labeled_data_path) # y

# 파일명 랜덤
random.shuffle(origin_data_list)
random.shuffle(labeled_data_list)

# train, test / x, y 
train_x_file = origin_data_list[:int(len(origin_data_list)*0.8)]
train_y_file = [file_name[:-4] + '_L.png' for file_name in train_x_file]

test_x_file = [file_name for file_name in origin_data_list if file_name not in train_x_file]
test_y_file = [file_name[:-4] + '_L.png' for file_name in test_x_file]

In [3]:
print(len(train_x_file), len(test_x_file), len(train_y_file), len(test_y_file))

560 141 560 141


In [4]:
print(train_x_file[0], test_x_file[0], train_y_file[0], test_y_file[0])

Seq05VD_f02220.png 0016E5_08107.png Seq05VD_f02220_L.png 0016E5_08107_L.png


### X

In [5]:
train_x = [np.array(Image.open(origin_data_path + train)) for train in train_x_file]
test_x = [np.array(Image.open(origin_data_path + test)) for test in test_x_file]

### Y

In [6]:
# color to label catId
color2label = { label.color   : label.id for label in labels}

In [7]:
train_y = []
for file_name in tqdm(train_y_file):
    image = np.array(Image.open(labeled_data_path + file_name))
    ret = [[color2label[tuple([r[0], r[1], r[2]])] 
            if tuple([r[0], r[1], r[2]]) in color2label else 11
            for r in row] 
           for row in image]
    train_y.append(ret)

100%|██████████| 560/560 [21:43<00:00,  2.33s/it]


In [8]:
test_y = []
for file_name in tqdm(test_y_file):
    image = np.array(Image.open(labeled_data_path + file_name))
    ret = [[color2label[tuple([r[0], r[1], r[2]])] 
            if tuple([r[0], r[1], r[2]]) in color2label else 11
            for r in row] 
           for row in image]
    test_y.append(ret)

100%|██████████| 141/141 [05:28<00:00,  2.33s/it]


In [9]:
np.array(train_y).shape, np.array(test_y).shape

((560, 720, 960), (141, 720, 960))

In [10]:
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

### data save

In [11]:
np.savez('data.npz', train_x=train_x, train_y=train_y, test_x=test_x, test_y=test_y)

# Model

### package load

In [1]:
import os
import sys
import time
import datetime
from tqdm import tqdm
import numpy as np
import warnings
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms.functional as TF

In [2]:
from model import convert_bn_to_instancenorm, convert_bn_to_evonorm, convert_bn_to_groupnorm, DeepLabHead, UNet
from helpers import AverageMeter, ProgressMeter, iouCalc, visim, vislbl
from labels import labels

### CPU or GPU
#### 아래 코드에서 True이면 GPU 사용, False 이면 CPU 사용됨.

In [3]:
USE_CUDA = torch.cuda.is_available() and True 
device = torch.device('cuda' if USE_CUDA else 'cpu')

### data load

In [4]:
npzfile = np.load('data.npz')

train_x = npzfile['train_x']
train_y = npzfile['train_y']
test_x = npzfile['test_x']
test_y = npzfile['test_y']

npzfile.close()

### DeepLab v3 ResNet50 

In [5]:
model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=False).to(device)
model.classifier = DeepLabHead(2048, 12).to(device) # 12 = class num

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2) # 파라미터 학습속도 조절

In [7]:
# Initialize metrics
best_miou = 0.0
metrics = {'train_loss' : [],
           'train_acc' : [],
           'test_acc' : [],
           'test_loss' : [],
           'miou' : []}
start_epoch = 0

### Label 

In [8]:
# Create list of class names
classLabels = []
for label in labels:
    if label.name not in classLabels:
        classLabels.append(label.name)
classLabels.append('void')

In [9]:
validClasses = list(np.unique([label.id for label in labels if label.id >= 0] + [11]))

In [10]:
len(classLabels), len(validClasses), classLabels[0], validClasses[0]

(12, 12, 'Pole', 0)

### train / validataion

In [11]:
train_X = torch.tensor(train_x[:int(train_x.shape[0]*0.875)], dtype=torch.float32)
train_Y = torch.tensor(train_y[:int(train_y.shape[0]*0.875)], dtype=torch.long)

train_data = torch.utils.data.TensorDataset(train_X.permute(dims=(0, 3, 1, 2)), train_Y)

train_data = torch.utils.data.DataLoader(train_data, batch_size=2, shuffle=True)

In [12]:
val_X = torch.tensor(train_x[int(train_x.shape[0]*0.875):], dtype=torch.float32)
val_Y = torch.tensor(train_y[int(train_x.shape[0]*0.875):], dtype=torch.long)

val_data = torch.utils.data.TensorDataset(val_X.permute(dims=(0, 3, 1, 2)), val_Y)

val_data = torch.utils.data.DataLoader(val_data, batch_size=2, shuffle=False)

In [13]:
dist = {i:(train_Y == i).sum().tolist() for i in range(12)}

In [14]:
weights = [1/dist[i] for i in range(12)]
total_weights = sum(weights)

In [15]:
class_weight = torch.FloatTensor([w/total_weights for w in weights]).to(device)
criterion = nn.CrossEntropyLoss(weight = class_weight, ignore_index=12) # weight 파라미터에 class_weight 추가

In [28]:
num_epoch = 5
res = train_X.shape[1] * train_X.shape[2]

In [17]:
train_X.shape[1] * train_X.shape[2], val_X.shape[1] * val_X.shape[2]

(691200, 691200)

In [29]:
for epoch in range(num_epoch):
    model.train()
    
    # batch_time = AverageMeter('Time', ':6.3f')
    # data_time = AverageMeter('Data', ':6.3f')
    loss_running = AverageMeter('Loss', ':.4e')
    acc_running = AverageMeter('Accuracy', ':.3f')  
    iou = iouCalc(classLabels, validClasses, voidClass = 11)
    progress = ProgressMeter(
        len(train_data),
        [loss_running, acc_running],
        prefix="Train, epoch: [{}]".format(epoch))

    batch_loss = 0.0
    for batch, (x, y) in enumerate(tqdm(train_data, total=len(train_data))):
        x = x.to(device)
        y = y.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward pass
        outputs = model(x)
        outputs = outputs['out']
        preds = torch.argmax(outputs, 1)
        
        # cross-entropy loss
        loss = criterion(outputs, y)

        # backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        bs = x.size(0)
        loss = loss.item()
        loss_running.update(loss, bs)
        corrects = torch.sum((preds == y) & (y != 12))
        
        nvoid = int((y==12).sum())
        acc = corrects.double()/(bs*res-nvoid)
        acc_running.update(acc, bs)
        
        # Calculate IoU scores of current batch
        iou.evaluateBatch(preds, y)
        
#         progress.display(batch)
        
     
    scheduler.step(loss_running.avg)
    miou = iou.outputScores()
    
    print('train epoch ', epoch+1)
    print('loss : {:.4f}   acc : {:.4f}   miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))
    
    # save checkpoint per epoch
    now = datetime.datetime.now()
    now_time = now.strftime('%y%m%d_%H:%M')
    
    # save path
    if not os.path.isdir(os.getcwd() + '/result'):
        os.makedirs(os.getcwd() + '/result')
    
    save_path = os.getcwd() + '/result/'
    
    # Save best model to file
    torch.save({
        'epoch' : epoch,
        'model_state_dict' : model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_miou': best_miou,
        'metrics': metrics,
        }, save_path + now_time + '_checkpoint.pth.tar')
    
    # Save best model to file
    if miou > best_miou:
        print('mIoU improved from {:.4f} to {:.4f}.'.format(best_miou, miou))
        best_miou = miou
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            }, save_path + now_time + '_best_weights.pth.tar')


100%|██████████| 245/245 [04:03<00:00,  1.01it/s]


classes           IoU
---------------------
Pole          : 0.373
SignSymbol    : 0.587
Bicyclist     : 0.667
Pedestrian    : 0.553
Building      : 0.844
Fence         : 0.741
Pavement      : 0.830
Road          : 0.950
Car           : 0.866
Sky           : 0.900
Tree          : 0.814
---------------------
Mean IoU      : 0.739
---------------------
train epoch  1
loss : 0.2871   acc : 0.9082   miou : 0.7388
mIoU improved from 0.7367 to 0.7388.


100%|██████████| 245/245 [04:05<00:00,  1.00s/it]


classes           IoU
---------------------
Pole          : 0.392
SignSymbol    : 0.612
Bicyclist     : 0.683
Pedestrian    : 0.589
Building      : 0.853
Fence         : 0.760
Pavement      : 0.842
Road          : 0.954
Car           : 0.875
Sky           : 0.904
Tree          : 0.824
---------------------
Mean IoU      : 0.753
---------------------
train epoch  2
loss : 0.2536   acc : 0.9141   miou : 0.7534
mIoU improved from 0.7388 to 0.7534.


100%|██████████| 245/245 [04:05<00:00,  1.00s/it]


...

mIoU improved from 0.7534 to 0.7779.


100%|██████████| 245/245 [04:05<00:00,  1.00s/it]


classes           IoU
---------------------
Pole          : 0.438
SignSymbol    : 0.677
Bicyclist     : 0.751
Pedestrian    : 0.647
Building      : 0.883
Fence         : 0.811
Pavement      : 0.865
Road          : 0.963
Car           : 0.896
Sky           : 0.911
Tree          : 0.848
---------------------
Mean IoU      : 0.790
---------------------
train epoch  4
loss : 0.1988   acc : 0.9289   miou : 0.7900
mIoU improved from 0.7779 to 0.7900.


100%|██████████| 245/245 [04:05<00:00,  1.00s/it]


classes           IoU
---------------------
Pole          : 0.456
SignSymbol    : 0.690
Bicyclist     : 0.759
Pedestrian    : 0.661
Building      : 0.890
Fence         : 0.817
Pavement      : 0.875
Road          : 0.966
Car           : 0.902
Sky           : 0.913
Tree          : 0.852
---------------------
Mean IoU      : 0.798
---------------------
train epoch  5
loss : 0.1880   acc : 0.9325   miou : 0.7982
mIoU improved from 0.7900 to 0.7982.


### test

In [30]:
X = torch.tensor(test_x, dtype=torch.float32)
Y = torch.tensor(test_y, dtype=torch.long)

data = torch.utils.data.TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)

test_data = torch.utils.data.DataLoader(data, batch_size=1, shuffle=True)

In [31]:
# Load best model

# save path
if not os.path.isdir(os.getcwd() + '/result'):
    os.makedirs(os.getcwd() + '/result')

save_path = os.getcwd() + '/result/'
result = sorted(os.listdir(save_path), reverse=True)

In [32]:
checkpoint = torch.load(save_path + result[1]) # 가장 최신 best_weights 파일 가져옴
model.load_state_dict(checkpoint['model_state_dict'], strict=True)
print('Loaded best model weights (epoch {}) from {}'.format(checkpoint['epoch'], save_path + result[1]))

Loaded best model weights (epoch 4) from /home/ag/Capstone2/TEST/result/210602_18:58_best_weights.pth.tar


In [33]:
batch_time = AverageMeter('Time', ':6.3f')
data_time = AverageMeter('Data', ':6.3f')
progress = ProgressMeter(
    len(test_data),
    [batch_time, data_time],
    prefix='Predict: ')

model.eval()

batch_loss = 0.0
for batch, (x, y) in enumerate(tqdm(test_data, total=len(test_data))):

    x = x.to(device)
    y = y.to(device)

    # forward
    outputs = model(x)
    outputs = outputs['out']

    preds = torch.argmax(outputs, 1)

    # cross-entropy loss
    loss = criterion(outputs, y)

    # Statistics
    bs = x.size(0)
    loss = loss.item()
    loss_running.update(loss, bs)
    corrects = torch.sum((preds == y) & (y != 12))

    nvoid = int((y==12).sum())
    acc = corrects.double()/(bs*res-nvoid)
    acc_running.update(acc, bs)

    # Calculate IoU scores of current batch
    iou.evaluateBatch(preds, y)

miou = iou.outputScores()
scheduler.step(loss_running.avg)

print('loss : {:.4f} acc : {:.4f} miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))

100%|██████████| 141/141 [00:15<00:00,  9.39it/s]

classes           IoU
---------------------
Pole          : 0.444
SignSymbol    : 0.655
Bicyclist     : 0.749
Pedestrian    : 0.642
Building      : 0.887
Fence         : 0.800
Pavement      : 0.866
Road          : 0.965
Car           : 0.896
Sky           : 0.914
Tree          : 0.846
---------------------
Mean IoU      : 0.788
---------------------
loss : 0.2247 acc : 0.9292 miou : 0.7876


In [ ]:
# validataion
model.eval()

loss_running = AverageMeter('Loss', ':.4e')
acc_running = AverageMeter('Accuracy', ':.3f')
iou = iouCalc(classLabels, validClasses, voidClass = 11)

batch_loss = 0.0
for batch, (x, y) in enumerate(tqdm(val_data, total=len(val_data))):

    x = x.to(device)
    y = y.to(device)

    # forward
    outputs = model(x)
    outputs = outputs['out']

    preds = torch.argmax(outputs, 1)

    # cross-entropy loss
    loss = criterion(outputs, y)

    # Statistics
    bs = x.size(0)
    loss = loss.item()
    loss_running.update(loss, bs)
    corrects = torch.sum((preds == y) & (y != 12))

    nvoid = int((y==12).sum())
    acc = corrects.double()/(bs*res-nvoid)
    acc_running.update(acc, bs)

    # Calculate IoU scores of current batch
    iou.evaluateBatch(preds, y)

miou = iou.outputScores()

# Reduce learning rate
scheduler.step(loss_running.avg) 

print('validataion')
print('loss : {:.4f}   acc : {:.4f}   miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))